# 1、获取大模型

In [1]:
#导入 dotenv 库的 load_dotenv 函数，用于加载环境变量文件（.env）中的配置
import dotenv
from langchain_openai import ChatOpenAI
import os

dotenv.load_dotenv()  #加载当前目录下的 .env 文件

os.environ['OPENAI_API_KEY'] = os.getenv("SILICON_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("SILICON_URL")

# 创建大模型实例
llm = ChatOpenAI(model="Qwen/Qwen3-8B")  # 默认使用 gpt-3.5-turbo

# 直接提供问题，并调用llm
response = llm.invoke("什么是大模型？")
print(response)

d:\Softwares\MiniConda\envs\langchain-tutorial\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


content='“大模型”是指在人工智能领域中，特别是自然语言处理（NLP）和机器学习中，参数量非常庞大的神经网络模型。这类模型通常包含数亿甚至数百亿个参数，具有强大的学习和推理能力，可以理解和生成自然语言、图像、音频、视频等多种类型的数据。\n\n### 一、大模型（Large Model）的定义\n大模型一般指的是在训练过程中使用海量数据，并通过大规模计算资源进行训练的深度学习模型。它们通常基于深度神经网络（如Transformer），并具有以下几个核心特征：\n\n1. **参数量大**：通常是数十亿到数千亿参数，远超传统小模型。\n2. **训练数据多**：使用大量的文本、图像、语音等数据进行训练。\n3. **计算资源需求高**：需要高性能的GPU/TPU集群，以及大量的计算时间。\n4. **能力强大**：能完成多种复杂的任务，如自然语言理解、文本生成、语音识别、图像识别等。\n5. **泛化能力好**：在未见过的数据上也能表现出较佳的性能，具有较好的迁移能力。\n\n---\n\n### 二、大模型的应用场景\n大模型因其强大的泛化能力和多模态处理能力，被广泛应用于多个领域，包括但不限于：\n\n- **自然语言处理（NLP）**：\n  - 文本生成（如文章、故事、对话）\n  - 问答系统（如Chatbot）\n  - 机器翻译\n  - 情感分析\n  - 文本摘要\n  - 拼写和语法纠错\n\n- **计算机视觉（CV）**：\n  - 图像分类\n  - 目标检测\n  - 图像生成\n  - 图像修复\n\n- **语音识别与生成**：\n  - 语音转文字\n  - 文字转语音\n  - 语音合成\n\n- **多模态任务**：\n  - 文字、图像、语音联合处理（如图文生成、视频分析）\n\n---\n\n### 三、大模型的代表\n目前比较知名的大模型有：\n\n- **自然语言处理**：\n  - GPT 系列（GPT-1, GPT-2, GPT-3, GPT-4）\n  - BERT、RoBERTa、T5\n  - Megatron、PaLM、Llama、Qwen（通义千问）、通义万相等\n\n- **图像处理**：\n  - DALL-E\n  - Stable Diffusion\n  - Midjourney\n\n- 

# 2、使用提示词模板

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 需要注意的一点是，这里需要指明具体的role，在这里是system和用户
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档编写者"),
    ("user", "{input}")  # {input}为变量
])

# 我们可以把prompt和具体llm的调用和在一起。
chain = prompt | llm
message = chain.invoke({"input": "大模型中的LangChain是什么?"})
print(message)

# print(type(message))

# 3、 使用输出解析器

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser

# 初始化模型
llm = ChatOpenAI(model="gpt-4o-mini")

# 创建提示模板
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是世界级的技术文档编写者。"),
    ("user", "{input}")
])

# 使用输出解析器
# output_parser = StrOutputParser()
output_parser = JsonOutputParser()

# 将其添加到上一个链中
# chain = prompt | llm
chain = prompt | llm | output_parser

# 调用它并提出同样的问题。答案是一个字符串，而不是ChatMessage
# chain.invoke({"input": "LangChain是什么?"})
chain.invoke({"input": "LangChain是什么? 用JSON格式回复，问题用question，回答用answer"})

# 4、使用向量存储

In [ ]:
# 导入和使用 WebBaseLoader
from langchain_community.document_loaders import WebBaseLoader
import bs4

loader = WebBaseLoader(
        web_path="https://www.gov.cn/yaowen/liebiao/202601/content_7054758.htm",
        bs_kwargs=dict(parse_only=bs4.SoupStrainer(id="UCAP-CONTENT"))
    )
docs = loader.load()
# print(docs)

# 对于嵌入模型，这里通过 API调用
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 使用分割器分割文档
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
documents = text_splitter.split_documents(docs)
print(len(documents))
# 向量存储  embeddings 会将 documents 中的每个文本片段转换为向量，并将这些向量存储在 FAISS 向量数据库中
vector = FAISS.from_documents(documents, embeddings)

# 5、RAG(检索增强生成)

In [ ]:
from langchain_core.prompts import PromptTemplate

retriever = vector.as_retriever()
retriever.search_kwargs = {"k": 3}
docs = retriever.invoke("建设用地使用权是什么？")

# for i,doc in enumerate(docs):
#     print(f"⭐第{i+1}条规定：")
#     print(doc)

# 6.定义提示词模版
prompt_template = """
你是一个问答机器人。
你的任务是根据下述给定的已知信息回答用户问题。
确保你的回复完全依据下述已知信息。不要编造答案。
如果下述已知信息不足以回答用户的问题，请直接回复"我无法回答您的问题"。

已知信息:
{info}

用户问：
{question}

请用中文回答用户问题。
"""
# 7.得到提示词模版对象
template = PromptTemplate.from_template(prompt_template)

# 8.得到提示词对象
prompt = template.format(info=docs, question='本次会议都有谁出席')

## 9. 调用LLM
response = llm.invoke(prompt)
print(response.content)

# 6、使用Agent

In [ ]:
from langchain.tools.retriever import create_retriever_tool

# 检索器工具
retriever_tool = create_retriever_tool(
    retriever,
    "CivilCodeRetriever",
    "搜索有关中华人民共和国民法典的信息。关于中华人民共和国民法典的任何问题，您必须使用此工具!",
)

tools = [retriever_tool]

from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# https://smith.langchain.com/hub
prompt = hub.pull("hwchase17/openai-functions-agent")

agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# 运行代理
agent_executor.invoke({"input": "建设用地使用权是什么"})